In [1]:
## import libraries

import sys
import os
import pandas as pd
import arcpy
from arcgis.features import GeoAccessor
from arcgis import GIS

In [2]:
## environment settings
## more info: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
pd.options.mode.copy_on_write = True

## To allow overwriting outputs change overwriteOutput option to True.
arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

## Check out any necessary licenses.
arcpy.CheckOutExtension("spatial")
arcpy.CheckOutExtension("ImageAnalyst")

'CheckedOut'

In [3]:

print (sys.argv)

val = input("Enter the year ypu want to process: ")
print(val)
# val = 2020

['c:\\Users\\keit8223\\AppData\\Local\\ESRI\\conda\\envs\\arcgispro-py3-clone\\Lib\\site-packages\\ipykernel_launcher.py', '--f=c:\\Users\\keit8223\\AppData\\Roaming\\jupyter\\runtime\\kernel-v31042749bb924fc2f7d1c1107c97c831948723940.json']
2017


In [4]:
## define variables
## connect to GIS
gis = GIS()

## directory where the daily fleet csv files from GFW are stored
analysis_directory = os.path.join('E:\\analysis\\GFW', str(val), 'TIF')
# print(analysis_directory)
directory = r'F:\Global Fishing Watch\Fishing Intensity\v2\fleet-daily\fleet-daily-csvs-100-v2-'+ str(val)
# gear_list = ['fishing', 'drifting_longlines', 'seiners', 'purse_seines', 'tuna_purse_seines', 'other_purse_seines', 'other_seines', 'trollers', 'fixed_gear', 'pots_and_traps', 'set_longlines', 'set_gillnets', 'dredge_fishing', 'squid_jiggers', 'other']
mosacic_name = 'global_fish_watch_daily_fleet_sum_' + str(val)
processing_gdb = r'E:\analysis\GFW\gfw_daily_fleet.gdb'
mosaic_path = os.path.join(processing_gdb, mosacic_name)
projectCoordinateSystem="GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]"

In [14]:
def create_directories():
    if not os.path.exists(analysis_directory):
        os.makedirs(analysis_directory)
    if not arcpy.Exists(processing_gdb):
        arcpy.management.CreateFileGDB(os.path.dirname(processing_gdb), os.path.basename(processing_gdb))


In [15]:
def preprocess():
            print("Starting processing of " + filename + "...")
            # Get the full path for the filename
            file_path = os.path.join(directory, filename)
            # Read the CSV file
            # print(file_path)
            global data
            data = pd.read_csv(file_path)
            # Get the first row as a table
            first_row = data.head(1)
            # Get the value in the first column
            global value
            value = first_row.iloc[0, 0]
            # replace the '-' with '_'
            value = value.replace('-', '_')
            global tif_name1
            tif_name1 = os.path.join(analysis_directory, "daily_fleet_" + value + "_hours.tif")
            global fc_hours
            fc_hours = os.path.join(processing_gdb, "daily_fleet_" + value + "_hours")
            global tif_name2
            tif_name2 = os.path.join(analysis_directory, "daily_fleet_" + value + "_fishhours.tif")
            data['cell_ll_lat'] = data['cell_ll_lat'] + 0.005
            data['cell_ll_lon'] = data['cell_ll_lon'] + 0.005
            data.rename(columns={'cell_ll_lat': 'lat'}, inplace=True)
            data.rename(columns={'cell_ll_lon': 'lon'}, inplace=True)
            # print("Done with " + value + "...")


In [16]:
def process_hours():
    hours_select = data.loc[data['hours'] != 0]
    hours_csv = os.path.join(analysis_directory, "daily_fleet_" + value + "_hours.csv")
    hours_select.drop(columns=['date', 'flag', 'geartype', 'fishing_hours', 'mmsi_present'], inplace=True)
    hours_select.to_csv(hours_csv, index=False)

In [17]:
def process_fishhours():
    fishhours_select = data.loc[data['fishing_hours'] != 0]
    fishhours_csv = os.path.join(analysis_directory, "daily_fleet_" + value + "_fishhours.csv")
    fishhours_select.drop(columns=['date', 'flag', 'geartype', 'hours', 'mmsi_present'], inplace=True)
    fishhours_select.to_csv(fishhours_csv, index=False)

In [ ]:
create_directories()

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        preprocess()
        process_hours()
        process_fishhours()
print("All done...")


In [ ]:
for filename in os.listdir(analysis_directory):
    if filename.endswith('fishhours.csv'):
        csv_name = (os.path.basename(filename))
        print("Processing " + csv_name + "...")
        fish_csv = (os.path.join(analysis_directory, os.path.splitext(filename)[0] + ".csv"))
        # print(fish_csv)
        fish_tif = (os.path.join(analysis_directory, os.path.splitext(filename)[0] + ".tif"))
        # print(fish_tif)
        fish_fc = (os.path.join(processing_gdb, os.path.splitext(os.path.basename(filename))[0]))
        # print(fish_fc)
        arcpy.management.XYTableToPoint(fish_csv, fish_fc, 'lon', 'lat', '', projectCoordinateSystem)
        arcpy.conversion.PointToRaster(fish_fc, 'fishing_hours', fish_tif, "SUM", 'fishing_hours', 0.01)

In [ ]:
for filename in os.listdir(analysis_directory):
    if filename.endswith('_hours.csv'):
        csv_name = (os.path.basename(filename))
        print("Processing " + csv_name + "...")
        hours_csv = (os.path.join(analysis_directory, os.path.splitext(filename)[0] + ".csv"))
        # print(hours_csv)
        hours_tif = (os.path.join(analysis_directory, os.path.splitext(filename)[0] + ".tif"))
        # print(hours_tif)
        hours_fc = (os.path.join(processing_gdb, os.path.splitext(os.path.basename(filename))[0]))
        # print(hours_fc)
        arcpy.management.XYTableToPoint(hours_csv, hours_fc, 'lon', 'lat', '', projectCoordinateSystem)
        arcpy.conversion.PointToRaster(hours_fc, 'hours', hours_tif, "SUM", 'hours', 0.01)

In [5]:
arcpy.management.CreateMosaicDataset(in_workspace = processing_gdb,
                                     in_mosaicdataset_name = mosacic_name,
                                     coordinate_system = projectCoordinateSystem)

<Result 'E:\\analysis\\GFW\\gfw_daily_fleet.gdb\\global_fish_watch_daily_fleet_sum_2017'>

In [6]:
## Add rasters to the mosaic dataset
arcpy.management.AddRastersToMosaicDataset(in_mosaic_dataset = mosaic_path,
                                           raster_type = "Raster Dataset",
                                           filter = "*.tif",
                                           input_path = analysis_directory,
                                           sub_folder="SUBFOLDERS")


<Result 'E:\\analysis\\GFW\\gfw_daily_fleet.gdb\\global_fish_watch_daily_fleet_sum_2017'>

In [7]:
## Calculate the date field
arcpy.management.CalculateField(
    in_table=mosaic_path,
    field="date",
    expression="""(!Name!.split('_')[3] + "-" + !Name!.split('_')[4] + "-" + !Name!.split('_')[2])""",
    expression_type="PYTHON3",
    code_block="",
    field_type="DATE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'E:\\analysis\\GFW\\gfw_daily_fleet.gdb\\global_fish_watch_daily_fleet_sum_2017'>

In [8]:
## Calculate the variable field
arcpy.management.CalculateField(
    in_table=mosaic_path,
    field="variable",
    expression="""(!Name!.split('_')[5].split('.')[0])""",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS")

<Result 'E:\\analysis\\GFW\\gfw_daily_fleet.gdb\\global_fish_watch_daily_fleet_sum_2017'>

In [9]:
## Build multidimensional info
arcpy.md.BuildMultidimensionalInfo(in_mosaic_dataset = mosaic_path,
                                   variable_field = 'variable',
                                   dimension_fields = 'date')

<Result 'E:\\analysis\\GFW\\gfw_daily_fleet.gdb\\global_fish_watch_daily_fleet_sum_2017'>

In [10]:
## Calculate statistics
with arcpy.EnvManager(parallelProcessingFactor="99%"):
    arcpy.management.CalculateStatistics(in_raster_dataset = mosaic_path)

In [11]:
## Set mosaic dataset properties
arcpy.management.SetMosaicDatasetProperties(
    in_mosaic_dataset=mosaic_path,
    rows_maximum_imagesize=15000,
    columns_maximum_imagesize=15000,
    allowed_compressions="LZ77;LERC",
    default_compression_type="LERC",
    JPEG_quality=85,
    LERC_Tolerance=0,
    resampling_type="NEAREST",
    clip_to_footprints="CLIP",
    footprints_may_contain_nodata="FOOTPRINTS_DO_NOT_CONTAIN_NODATA",
    clip_to_boundary="NOT_CLIP",
    color_correction="NOT_APPLY",
    allowed_mensuration_capabilities="Basic",
    default_mensuration_capabilities="Basic",
    allowed_mosaic_methods="ByAttribute;NorthWest;Center;LockRaster;Nadir;Viewpoint;Seamline;None",
    default_mosaic_method="ByAttribute",
    order_field="StdTime",
    order_base="YYYY/MM/DD",
    sorting_order="DESCENDING",
    mosaic_operator="FIRST",
    blend_width=10,
    view_point_x=600,
    view_point_y=300,
    max_num_per_mosaic=500,
    cell_size_tolerance=0.8,
    cell_size="0 0",
    metadata_level="FULL",
    transmission_fields="Name;MinPS;MaxPS;LowPS;HighPS;Tag;GroupName;ProductName;CenterX;CenterY;ZOrder;Shape_Length;Shape_Area;date;variable;StdTime;Dimensions;NONE",
    use_time="ENABLED",
    start_time_field="StdTime",
    end_time_field="StdTime",
    time_format="YYYY-MM-DD",
    geographic_transform=None,
    max_num_of_download_items=20,
    max_num_of_records_returned=1000,
    data_source_type="SCIENTIFIC",
    minimum_pixel_contribution=1,
    processing_templates="None;fishhours;hours",
    default_processing_template="None",
    time_interval=1,
    time_interval_units="Days",
    product_definition="NONE",
    product_band_definitions=None
)

<Result 'E:\\analysis\\GFW\\gfw_daily_fleet.gdb\\global_fish_watch_daily_fleet_sum_2017'>

In [12]:
## Copy raster to CRF
with arcpy.EnvManager(compression="'LERC' 0.000000", parallelProcessingFactor="99%", pyramid="PYRAMIDS -1 NEAREST LZ77 85 NO_SKIP NO_SIPS"):
    arcpy.management.CopyRaster(
        in_raster=mosaic_path,
        out_rasterdataset=os.path.join(analysis_directory, mosacic_name + ".crf"),
        config_keyword="",
        background_value=None,
        nodata_value="",
        onebit_to_eightbit="NONE",
        colormap_to_RGB="NONE",
        pixel_type="",
        scale_pixel_value="NONE",
        RGB_to_Colormap="NONE",
        format="CRF",
        transform="NONE",
        process_as_multidimensional="ALL_SLICES",
        build_multidimensional_transpose="TRANSPOSE"
        # build_multidimensional_transpose="NO_TRANSPOSE"
    )

In [13]:
## Aggregate the daily multidimensional raster to monthly SUM for each variable (fishhours and hours)
with arcpy.EnvManager(compression="'LERC' 0.000000", parallelProcessingFactor="99%", pyramid="PYRAMIDS -1 NEAREST LZ77 85 NO_SKIP NO_SIPS", scratchWorkspace=analysis_directory):
    out_multidimensional_raster = arcpy.ia.AggregateMultidimensionalRaster(
        in_multidimensional_raster=os.path.join(analysis_directory, mosacic_name + ".crf"),
        dimension="StdTime",
        aggregation_method="SUM",
        variables="fishhours;hours",
        aggregation_def="INTERVAL_KEYWORD",
        interval_keyword="MONTHLY",
        interval_value=None,
        interval_unit="",
        interval_ranges=None,
        aggregation_function="",
        ignore_nodata="DATA",
        dimensionless="DIMENSIONS",
        percentile_value=90,
        percentile_interpolation_type="NEAREST"
    )
    out_multidimensional_raster.save(os.path.join(analysis_directory, mosacic_name.replace('daily', 'monthly') + ".crf"))

In [14]:
## Aggregate the daily multidimensional raster to annual SUM for each variable (fishhours and hours)
with arcpy.EnvManager(compression="'LERC' 0.000000", parallelProcessingFactor="99%", pyramid="PYRAMIDS -1 NEAREST LZ77 85 NO_SKIP NO_SIPS", scratchWorkspace=analysis_directory):
    out_multidimensional_raster = arcpy.ia.AggregateMultidimensionalRaster(
        in_multidimensional_raster=os.path.join(analysis_directory, mosacic_name + ".crf"),
        dimension="StdTime",
        aggregation_method="SUM",
        variables="fishhours;hours",
        aggregation_def="ALL",
        interval_keyword="",
        interval_value=None,
        interval_unit="",
        interval_ranges=None,
        aggregation_function="",
        ignore_nodata="DATA",
        dimensionless="DIMENSIONS",
        percentile_value=90,
        percentile_interpolation_type="NEAREST"
    )
    out_multidimensional_raster.save(os.path.join(analysis_directory, mosacic_name.replace('daily', 'annual') + ".crf"))

In [15]:
##########################################################
## Try to publish the layers to AGOL ##
##########################################################



In [16]:
# import arcgis
# from arcgis.raster import *
# from arcgis.raster.analytics import *
# from arcgis.gis import GIS
# # from AGOL_Credentials import AGOLusername, AGOLpassword

# arcgis.env.verbose=True
# # username = AGOLusername
# # password = AGOLpassword
# gis = GIS()

# ## https://developers.arcgis.com/python/api-reference/arcgis.raster.analytics.html
# import os

# generate_raster_op = copy_raster(input_raster = os.path.join(analysis_directory, mosacic_name.replace('daily', 'annual') + ".crf"),
#                                      output_name = mosacic_name.replace('daily', 'annual'),
#                                      process_as_multidimensional=True,
#                                      raster_type_name="Raster Dataset",
#                                      tiles_only=True,
#                                      gis=gis)